In [ ]:
import numpy as np
import ast
import os
import datetime as dt
import sys
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def get_flat(x, flat):
    # Use list comprehension for efficiency
    flat.extend(ast.literal_eval(x))

def get_flat_timestamp(x, flat, fs=100):
    format1 = "%Y-%m-%d %H:%M:%S.%f%z"
    format2 = "%Y-%m-%d %H:%M:%S%z"
    
    # Convert timestamp to datetime once
    try:
        start_time_converted = dt.datetime.strptime(x, format1)
    except ValueError:
        start_time_converted = dt.datetime.strptime(x, format2)
    
    # Generate list of timestamps using NumPy for efficiency
    time_deltas = np.arange(fs) * (1 / fs)  # create array of time deltas
    dt_list = start_time_converted + np.array([dt.timedelta(seconds=td) for td in time_deltas])
    
    flat.extend(dt_list)

def safe_get_flat_ecg_timestamp(row, shifted_series, flat, fs=100):
    try:
        next_x = shifted_series.loc[row.name]
        get_flat_ecg_timestamp(row['timestamp'], next_x['timestamp'], flat, fs)
    except Exception:
        get_flat_timestamp(row['timestamp'], flat, fs)

def get_flat_ecg_timestamp(x, next_x, flat, fs=100):
    format1 = "%Y-%m-%d %H:%M:%S.%f%z"
    format2 = "%Y-%m-%d %H:%M:%S%z"
    
    # Parse timestamps once
    try:
        start_time_converted = dt.datetime.strptime(x, format1)
        end_time_converted = dt.datetime.strptime(next_x, format1)
    except ValueError:
        start_time_converted = dt.datetime.strptime(x, format2)
        end_time_converted = dt.datetime.strptime(next_x, format2)

    # Calculate the number of samples and time intervals using NumPy
    total_duration = (end_time_converted - start_time_converted).total_seconds()
    num_samples = int(total_duration * fs)
    time_deltas = np.arange(num_samples) * (1 / fs)  # array of time deltas

    dt_list = start_time_converted + np.array([dt.timedelta(seconds=td) for td in time_deltas])
    flat.extend(dt_list)


In [ ]:
# Load data
PPG_PATH = os.path.join("..","..","sample_data", "ecg.csv")
df_ppg = pd.read_csv(PPG_PATH)

# Initialize empty lists to collect data
signal_col = []
df_ppg['ecg'].apply(get_flat, flat=signal_col)

date_col = []
fs = 100
df_ppg['timestamp'].apply(get_flat_timestamp, flat=date_col, fs=fs)

fig = go.Figure()
fig.add_trace(go.Scatter(y=signal_col ,mode="lines"))
fig.show()

In [ ]:
!pip install -e "../../."

In [ ]:
import os,sys
sys.path.append("..")
from vitalDSP.visualization.filtering_visualization import FilteringVisualization

filter = FilteringVisualization(signal_col)
FilteringVisualization.visualize_all_filters()
